# [Lv4-Day4-Lab3] The Asynchronous Orchestra: An Event-Driven Agent Collaboration Protocol

### 실습 목표
4일차 세 번째 실습에서는, Lab 2에서 구축한 '동기식(Synchronous)' 오케스트레이션의 한계를 극복하고, 여러 Agent가 **'비동기적(Asynchronous)'**으로 협업하는 고도로 확장 가능한 시스템을 밑바닥부터 구축합니다. 우리는 Agent가 다른 Agent를 직접 호출하는 방식에서 벗어나, 모든 작업을 **'메시지 큐(Message Queue)'**를 통해 주고받는 **이벤트 기반 아키텍처(Event-Driven Architecture)**를 구현합니다.

### 0. 사전 준비: 라이브러리 설치 및 Upstash Redis 연결
이 실습은 3일차 Lab 3에서 사용했던 Upstash Redis 데이터베이스를 통신 채널로 사용합니다.

In [ ]:
# !pip install redis tavily-python -q

In [ ]:
import os
import redis
import time
import threading
import json
from getpass import getpass
from tavily import TavilyClient

# --- Upstash Redis 연결 설정 ---
redis_client = None

UPSTASH_URL = getpass("UPSTASH_URL 을 입력하세요: ")
UPSTASH_TOKEN = getpass("UPSTASH_REDIS_REST_TOKEN 을 입력하세요: ")
redis_client = redis.Redis.from_url(
    f"rediss://:{UPSTASH_TOKEN}@{UPSTASH_URL.replace('https://', '')}", decode_responses=True
)
redis_client.ping()
print("✅ Upstash Redis 서버에 성공적으로 연결되었습니다.")

# --- 검색 도구(Tavily) 준비 ---
TAVILY_API_KEY = getpass("TAVILY_API_KEY 를 입력하세요: ")
if TAVILY_API_KEY:
    tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
    # 간단한 테스트 검색으로 키 유효성 확인
    tavily_client.search(query="Tavily API test", max_results=1)
    print("✅ Tavily 검색 클라이언트가 성공적으로 준비되었습니다.")

### 1. Subscriber (구독자) 구현: '작업 대기 중인 Worker Agent' 시뮬레이션

먼저, 메시지 큐에 새로운 작업이 들어오기를 기다리는 **'구독자(Subscriber)'** 로직을 구현합니다. 이는 우리 시스템에서 **'Worker Agent'**의 역할을 시뮬레이션합니다. Worker Agent는 특정 채널(예: `task-requests`)을 구독하고 있다가, 새로운 작업 메시지가 도착하면 즉시 이를 인지하고 처리할 수 있습니다.

**기술적 핵심: `threading`**
구독자의 `listen()` 메소드는 새로운 메시지가 올 때까지 무한정 대기(Blocking)합니다. 만약 이 코드를 메인 스레드에서 실행하면 Jupyter Notebook 전체가 멈추게 됩니다. 이를 해결하기 위해, 우리는 **별도의 스레드(Thread)**에서 구독자 로직을 실행하여, 노트북의 다른 작업을 방해하지 않으면서도 백그라운드에서 계속 메시지를 기다리도록 만듭니다. 이는 실무에서 여러 프로세스를 동시에 관리하는 방식의 기초입니다.

In [ ]:
def subscriber_worker(channel_name: str):
    """특정 채널을 구독하고 메시지를 수신하는 워커 함수"""
    print(f"[Subscriber] '{channel_name}' 채널 구독을 시작합니다. (백그라운드 실행 중...)")

    # 각 스레드는 자체 Redis 클라이언트 인스턴스를 사용하는 것이 안전합니다.
    sub_client = redis.Redis.from_url(
        f"rediss://:{UPSTASH_TOKEN}@{UPSTASH_URL.replace('https://', '')}", decode_responses=True
    )
    pubsub = sub_client.pubsub()
    pubsub.subscribe(channel_name)

    try:
        for message in pubsub.listen():
            if message["type"] == "message":
                data = json.loads(message["data"])
                print(
                    f"\n[Subscriber] 📬 메시지 수신! (채널: {message['channel']})\n    - Task ID: {data['task_id']}\n    - 내용: {data['payload']}"
                )
    except redis.exceptions.ConnectionError:
        print("[Subscriber] Redis 연결이 종료되었습니다.")
    except Exception as e:
        print(f"[Subscriber] 오류 발생: {e}")
    finally:
        pubsub.close()
        print("[Subscriber] 구독을 종료합니다.")


print("✅ Subscriber 워커 함수가 정의되었습니다.")

### 2. Publisher (발행자) 구현 및 Pub/Sub 시스템 시뮬레이션

이제 새로운 작업을 생성하여 메시지 큐에 던져주는 **'발행자(Publisher)'** 로직을 구현합니다. 이는 우리 시스템에서 **'Dispatcher'**의 역할을 시뮬레이션합니다. Publisher는 어떤 Subscriber가 메시지를 받을지 전혀 신경 쓰지 않고, 오직 정해진 채널에 메시지를 '발행'하기만 합니다. 이것이 바로 시스템을 유연하게 만드는 **'디커플링(Decoupling)'**의 힘입니다.

아래 셀을 실행하면, 먼저 **백그라운드에서 Subscriber 스레드가 시작**되어 메시지를 기다립니다. 그 후, 메인 스레드에서 **Publisher가 3개의 서로 다른 작업 메시지를 순차적으로 발행**하면, Subscriber가 이를 실시간으로 수신하여 출력하는 것을 확인할 수 있습니다.

In [ ]:
if redis_client:
    TASK_CHANNEL = "agent:task_requests"

    # 1. Subscriber 스레드 시작
    subscriber_thread = threading.Thread(
        target=subscriber_worker, args=(TASK_CHANNEL,), daemon=True  # 메인 프로그램 종료 시 스레드도 함께 종료
    )
    subscriber_thread.start()
    time.sleep(2)  # 구독자가 채널에 연결될 시간을 줌

    # 2. Publisher 로직 실행 (Dispatcher 역할)
    print(f"\n[Publisher] '{TASK_CHANNEL}' 채널로 3개의 작업 메시지를 발행합니다...")

    # 작업 1: 리서치
    task_1 = {"task_id": "task_001", "payload": "'양자 컴퓨팅'에 대한 자료 조사"}
    redis_client.publish(TASK_CHANNEL, json.dumps(task_1))
    print(f"  - 📤 Task 1 발행 완료")
    time.sleep(1)

    # 작업 2: 초안 작성
    task_2 = {"task_id": "task_002", "payload": "Task 1의 결과를 바탕으로 초안 작성"}
    redis_client.publish(TASK_CHANNEL, json.dumps(task_2))
    print(f"  - 📤 Task 2 발행 완료")
    time.sleep(1)

    # 작업 3: 품질 검증
    task_3 = {"task_id": "task_003", "payload": "Task 2의 초안 검토 및 편집"}
    redis_client.publish(TASK_CHANNEL, json.dumps(task_3))
    print(f"  - 📤 Task 3 발행 완료")

    # 시뮬레이션 종료를 위해 잠시 대기 후 스레드 정리 (실제 시스템에서는 계속 실행됨)
    time.sleep(2)
    print("\n[Publisher] 모든 메시지 발행 완료. 시뮬레이션을 종료합니다.")

else:
    print("❌ Redis 클라이언트가 초기화되지 않았습니다. 이전 셀을 다시 실행해주세요.")

### 3. 시스템의 '기억': Task Registry 구현

Part 1에서 구축한 Pub/Sub 시스템은 메시지를 전달할 뿐, 작업의 **'상태'**를 추적하지는 않습니다. Worker Agent가 작업을 받다가 실패하면, 그 작업은 영원히 유실될 수 있습니다. 이러한 문제를 해결하기 위해, 우리는 모든 작업의 생성부터 완료까지 전 과정을 기록하고 추적하는 **'Task Registry'**를 구현합니다.

**Why Redis `HASH`?**
우리는 Redis의 `HASH` 자료구조를 사용하여 Task Registry를 구축합니다. `HASH`는 하나의 키(우리의 경우 `task_id`) 아래에 여러 개의 필드(field)와 값(value)을 저장할 수 있어, 마치 Python의 딕셔너리처럼 구조화된 데이터를 효율적으로 관리하는 데 최적화되어 있습니다.

**구현 목표:**
1.  **Task 모델 정의:** Pydantic을 사용하여 Task의 상태(`PENDING`, `RUNNING`, `COMPLETED`, `FAILED`), 입력값, 결과물 등 모든 정보를 담는 명확한 데이터 '설계도'를 만듭니다.
2.  **`TaskRegistry` 클래스 구현:** Task를 생성하고, 상태를 업데이트하며, 특정 Task의 정보를 조회하는 모든 로직을 캡슐화한 전문 클래스를 설계합니다.
3.  **상태 전이(State Transition) 시뮬레이션:** Task가 '대기' -> '진행 중' -> '완료' 상태로 변화하는 과정을 시뮬레이션하고, 각 단계마다 Redis에 저장된 실제 데이터가 어떻게 변경되는지 직접 확인합니다.

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any
from enum import Enum
import uuid
import datetime
import json


# 1. Task의 상태를 나타내는 Enum 정의
class TaskStatus(str, Enum):
    PENDING = "PENDING"
    RUNNING = "RUNNING"
    COMPLETED = "COMPLETED"
    FAILED = "FAILED"


# 2. Task 데이터 구조를 위한 Pydantic 모델 정의
class Task(BaseModel):
    task_id: str = Field(default_factory=lambda: f"task_{uuid.uuid4().hex[:8]}")
    status: TaskStatus = TaskStatus.PENDING
    payload: Dict[str, Any]
    priority: int = Field(default=5, description="작업 우선순위 (1-10, 낮을수록 높음)")
    result: Optional[str] = None
    worker_id: Optional[str] = None
    created_at: str = Field(default_factory=lambda: datetime.datetime.now(datetime.timezone.utc).isoformat())
    updated_at: str = Field(default_factory=lambda: datetime.datetime.now(datetime.timezone.utc).isoformat())


# 3. Task Registry 클래스 구현
class TaskRegistry:
    def __init__(self, client: redis.Redis):
        self._client = client

    def _get_task_key(self, task_id: str) -> str:
        return f"task_registry:{task_id}"

    def _serialize_task_data(self, task: Task) -> Dict[str, str]:
        """Task 데이터를 Redis에 저장 가능한 형태로 직렬화"""
        data = task.model_dump()
        # payload를 JSON 문자열로 변환
        data["payload"] = json.dumps(data["payload"])
        return {k: str(v) for k, v in data.items()}

    def _deserialize_task_data(self, data: Dict[str, str]) -> Task:
        """Redis에서 가져온 데이터를 Task 객체로 역직렬화"""
        if "payload" in data:
            data["payload"] = json.loads(data["payload"])
        return Task(**data)

    def register_task(self, payload: Dict[str, Any], priority: int = 5) -> Task:
        """새로운 Task를 생성하고 Registry에 등록합니다."""
        new_task = Task(payload=payload, priority=priority)
        key = self._get_task_key(new_task.task_id)
        serialized_data = self._serialize_task_data(new_task)
        self._client.hset(key, mapping=serialized_data)
        print(f"[Registry] ✅ Task 등록됨: {new_task.task_id} (Priority: {priority}, Status: {new_task.status})")
        return new_task

    def update_task_status(
        self,
        task_id: str,
        new_status: TaskStatus,
        worker_id: Optional[str] = None,
        result: Optional[str] = None,
        priority: Optional[int] = None,
    ):
        """Task의 상태, 담당 워커, 결과 등을 업데이트합니다."""
        key = self._get_task_key(task_id)
        if not self._client.exists(key):
            print(f"[Registry] 🔴 오류: Task를 찾을 수 없음 - {task_id}")
            return

        update_data = {
            "status": new_status.value,
            "updated_at": datetime.datetime.now(datetime.timezone.utc).isoformat(),
        }
        if worker_id:
            update_data["worker_id"] = worker_id
        if result:
            update_data["result"] = result
        if priority is not None:
            update_data["priority"] = str(priority)

        self._client.hset(key, mapping=update_data)
        print(f"[Registry] 🔄 Task 업데이트됨: {task_id} (New Status: {new_status.value})")

    def get_task(self, task_id: str) -> Optional[Task]:
        """특정 Task의 모든 정보를 가져옵니다."""
        key = self._get_task_key(task_id)
        task_data = self._client.hgetall(key)
        if not task_data:
            return None
        return self._deserialize_task_data(task_data)


# Task Registry 인스턴스 생성
if "redis_client" in locals() and redis_client:
    try:
        redis_client.ping()
        task_registry = TaskRegistry(redis_client)
        print("\n✅ TaskRegistry가 성공적으로 초기화되었습니다.")
    except Exception as e:
        print(f"\n❌ Redis 연결 오류: {e}")
else:
    print("\n❌ Redis 클라이언트가 초기화되지 않았습니다. 이전 셀을 다시 실행해주세요.")

### 4. Task Registry 시뮬레이션: '작업의 일생' 추적하기

이제 `TaskRegistry`가 의도대로 잘 작동하는지, 하나의 Task가 생성되어 완료되기까지의 전체 라이프사이클을 시뮬레이션하며 확인합니다. 각 단계를 거칠 때마다, Redis에 저장된 실제 `HASH` 데이터가 어떻게 변화하는지 직접 눈으로 관찰하여 시스템의 작동 원리를 명확히 이해합니다.

In [ ]:
if "task_registry" in locals():
    print("--- 🚀 Task 라이프사이클 시뮬레이션 시작 ---")

    try:
        # 1. Dispatcher가 새로운 Task를 등록
        payload_data = {"topic": "LangGraph와 CrewAI의 차이점", "user_id": "user_123"}
        new_task = task_registry.register_task(payload=payload_data, priority=3)
        task_id = new_task.task_id

        print("\n--- [Redis 데이터 확인 1: PENDING] ---")
        stored_data = redis_client.hgetall(f"task_registry:{task_id}")
        print(json.dumps(stored_data, indent=2, ensure_ascii=False))

        # 2. Worker Agent가 Task를 가져가서 상태를 'RUNNING'으로 업데이트
        time.sleep(1)
        worker_id = "crewai_worker_01"
        task_registry.update_task_status(task_id, TaskStatus.RUNNING, worker_id=worker_id)

        print("\n--- [Redis 데이터 확인 2: RUNNING] ---")
        stored_data = redis_client.hgetall(f"task_registry:{task_id}")
        print(json.dumps(stored_data, indent=2, ensure_ascii=False))

        # 3. Worker Agent가 작업을 완료하고 상태를 'COMPLETED'로 업데이트
        time.sleep(2)
        result_data = "LangGraph는 유연한 제어 흐름에, CrewAI는 빠른 팀 구성에 강점이 있습니다..."
        task_registry.update_task_status(task_id, TaskStatus.COMPLETED, result=result_data)

        print("\n--- [Redis 데이터 확인 3: COMPLETED] ---")
        final_task_data = task_registry.get_task(task_id)
        if final_task_data:
            print(final_task_data.model_dump_json(indent=2))
        else:
            print("Task 데이터를 가져올 수 없습니다.")

        print("\n--- ✅ 시뮬레이션 완료 --- ")

    except Exception as e:
        print(f"❌ 시뮬레이션 중 오류 발생: {e}")
        import traceback

        traceback.print_exc()
else:
    print("❌ Task Registry가 초기화되지 않았습니다. 이전 셀을 다시 실행해주세요.")

### 5. 시스템의 '행위자': Worker Agent 구현

이제 우리 시스템의 실제 '일꾼'인 **Worker Agent**를 구현합니다. 각 Worker는 독립적인 개체로서, 자신만의 전문 분야(구독할 채널)를 가지고 있으며, 중앙 메시지 큐에 새로운 작업이 들어오기를 끊임없이 기다립니다.

**Worker의 작업 흐름:**
1.  **Listen (듣기):** 담당 채널을 구독하고 새로운 작업 메시지를 기다립니다 (`pubsub.listen()`).
2.  **Acknowledge (인지):** 메시지를 받으면, `task_id`를 추출하여 Task Registry에서 상세 정보를 가져옵니다.
3.  **Update (상태 변경):** Task의 상태를 `PENDING`에서 `RUNNING`으로 변경하여, 다른 Worker가 동일한 작업을 중복으로 수행하는 것을 방지합니다.
4.  **Execute (실행):** 실제 작업(LLM 호출, Tool 사용 등)을 수행합니다. (이번 파트에서는 `time.sleep`으로 시뮬레이션)
5.  **Complete (완료):** 작업이 끝나면, 결과와 함께 Task의 상태를 `COMPLETED` 또는 `FAILED`로 최종 업데이트합니다.

이 모든 과정은 **별도의 스레드**에서 실행되어, 여러 Worker Agent가 동시에, 그리고 비동기적으로 작동하는 환경을 시뮬레이션합니다.

In [ ]:
import threading
import time
import json


class WorkerAgent:
    def __init__(self, worker_id: str, specialty: str, registry: "TaskRegistry", redis_conn):
        self.worker_id = worker_id
        self.specialty = specialty
        self.registry = registry
        # 각 워커는 자체 Redis 연결을 사용
        self.redis_conn = redis.Redis.from_url(
            f"rediss://:{UPSTASH_TOKEN}@{UPSTASH_URL.replace('https://', '')}", decode_responses=True
        )
        self.stop_event = threading.Event()
        self.thread = threading.Thread(target=self.run, daemon=True)

    def execute_task(self, task: "Task") -> str:
        """
        실제 작업 로직을 수행하는 메소드.
        하위 클래스에서 이 메소드를 오버라이드하여 전문화된 작업을 구현합니다.
        """
        print(f"[{self.worker_id}] ⚙️ 기본 작업 실행 중... (내용: {task.payload})")
        return f"기본 작업 완료: {json.dumps(task.payload)}"

    def run(self):
        print(f"[{self.worker_id}] 🚀 워커 시작. '{self.specialty}' 채널을 구독합니다.")
        pubsub = self.redis_conn.pubsub()
        pubsub.subscribe(self.specialty)

        try:
            while not self.stop_event.is_set():
                message = pubsub.get_message(ignore_subscribe_messages=True, timeout=1.0)
                if message and message["data"]:
                    try:
                        task_notification = json.loads(message["data"])
                        task_id = task_notification["task_id"]

                        lock_key = f"task_lock:{task_id}"
                        if not self.redis_conn.setnx(lock_key, self.worker_id):
                            # print(f"[{self.worker_id}] ⏭️ Task {task_id} 이미 다른 워커가 처리 중")
                            continue
                        self.redis_conn.expire(lock_key, 60)

                        print(f"[{self.worker_id}] 📬 새로운 작업 수신: {task_id}")

                        self.registry.update_task_status(task_id, TaskStatus.RUNNING, worker_id=self.worker_id)

                        task = self.registry.get_task(task_id)
                        if task:

                            result = self.execute_task(task)
                            final_status = TaskStatus.COMPLETED
                        else:
                            result = "Task를 찾을 수 없음"
                            final_status = TaskStatus.FAILED

                        self.registry.update_task_status(task_id, final_status, result=result)
                        self.redis_conn.delete(lock_key)  # 작업 완료 후 잠금 해제

                    except json.JSONDecodeError as e:
                        print(f"[{self.worker_id}] ❌ 메시지 파싱 오류: {e}")
                    except Exception as e:
                        print(f"[{self.worker_id}] ❌ 작업 처리 중 오류: {e}")
                        if "task_id" in locals():
                            self.registry.update_task_status(task_id, TaskStatus.FAILED, result=str(e))

                time.sleep(0.1)

        except Exception as e:
            print(f"[{self.worker_id}] ❌ 워커 실행 중 오류: {e}")
        finally:
            pubsub.close()
            print(f"[{self.worker_id}] 🛑 워커 종료.")

    def start(self):
        self.thread.start()

    def stop(self):
        self.stop_event.set()
        self.thread.join(timeout=5)


print("✅ WorkerAgent 클래스가 성공적으로 정의되었습니다. ")

### 6. 시스템의 '지휘자': Task Dispatcher 구현 및 전체 시스템 시뮬레이션

이제 시스템의 시작점인 **`TaskDispatcher`**를 구현합니다. Dispatcher는 외부로부터의 작업 요청을 받아, 이를 공식적인 `Task`로 만들어 `TaskRegistry`에 등록하고, 관련 전문 채널에 **'새로운 작업이 있다'**는 알림 메시지를 발행하는 역할을 합니다.

아래 셀에서는 다음과 같은 전체 시스템의 흐름을 시뮬레이션합니다:
1.  **'리서처'**와 **'작가'**라는 두 명의 전문 Worker Agent를 생성하고, 각각 `research_tasks`와 `writing_tasks` 채널을 구독하도록 백그라운드에서 실행시킵니다.
2.  Dispatcher가 두 개의 서로 다른 작업을 생성하여 각각의 전문 채널로 발행합니다.
3.  각 Worker가 자신의 전문 분야에 맞는 작업을 어떻게 독립적으로 가져가서 처리하는지, 그리고 그 모든 과정이 `TaskRegistry`에 어떻게 실시간으로 기록되는지 관찰합니다.

In [ ]:
class TaskDispatcher:
    def __init__(self, registry: "TaskRegistry", redis_conn):
        self.registry = registry
        self.redis_conn = redis_conn

    def dispatch_task(self, channel: str, payload: Dict[str, Any], priority: int = 5):
        print(f"\n[Dispatcher] 🚚 '{channel}' 채널로 새로운 작업을 전달합니다...")
        task = self.registry.register_task(payload=payload, priority=priority)
        notification = {"task_id": task.task_id}
        self.redis_conn.publish(channel, json.dumps(notification))
        print(f"[Dispatcher] 📤 Task 알림 발행 완료: {task.task_id}")
        return task.task_id


# --- '실제' 작업을 수행하는 전문 Agent 클래스 정의 ---
class ResearcherAgent(WorkerAgent):
    """Tavily 검색을 통해 정보를 수집하는 리서처 에이전트"""

    def execute_task(self, task: "Task") -> str:
        # tavily_client가 성공적으로 초기화되었는지 확인
        if not "tavily_client" in globals() or not tavily_client:
            raise Exception("Tavily 검색 클라이언트가 준비되지 않았습니다. 이전 셀을 확인하세요.")

        topic = task.payload.get("topic")
        if not topic:
            raise ValueError("오류: 검색할 'topic'이 payload에 없습니다.")

        print(f"[{self.worker_id}] 🔍 '{topic}'에 대한 실제 웹 검색을 시작합니다...")
        try:
            # tavily_client를 직접 사용하여 실제 검색 수행
            search_result = tavily_client.search(query=topic, max_results=3, search_depth="advanced")

            # Tavily의 실제 결과(딕셔너리)를 직접 파싱하여 문자열로 만듭니다.
            # search_result['results']는 딕셔너리의 리스트입니다.
            snippets = [
                f"URL: {res['url']}\nTitle: {res['title']}\nContent: {res['content']}"
                for res in search_result.get("results", [])
            ]

            if not snippets:
                return f"'{topic}'에 대한 검색 결과를 찾을 수 없습니다."

            result_str = f"'{topic}'에 대한 검색 결과 요약 (상위 {len(snippets)}개):\n\n"
            result_str += "\n\n---\n\n".join(snippets)  # 각 결과를 명확히 구분

            print(f"[{self.worker_id}] ✅ 검색 완료. 결과물 생성.")
            return result_str
        except Exception as e:
            print(f"[{self.worker_id}] ❌ 검색 중 오류 발생: {e}")
            raise e


class WriterAgent(WorkerAgent):
    """이전 작업의 결과를 바탕으로 글을 작성하는 작가 에이전트"""

    def execute_task(self, task: "Task") -> str:
        source_task_id = task.payload.get("source_task_id")
        style = task.payload.get("style", "일반적인")
        if not source_task_id:
            raise ValueError("오류: 참조할 'source_task_id'가 payload에 없습니다.")

        print(f"[{self.worker_id}] 📄 Task '{source_task_id}'의 결과를 바탕으로 글 작성을 시작합니다...")

        source_task = None
        for _ in range(5):
            source_task = self.registry.get_task(source_task_id)
            if source_task and source_task.status == TaskStatus.COMPLETED:
                break
            if source_task and source_task.status == TaskStatus.FAILED:
                raise Exception(f"소스 작업({source_task_id})이 실패하여 글을 작성할 수 없습니다.")
            time.sleep(2)

        if not source_task or source_task.status != TaskStatus.COMPLETED:
            raise Exception(f"오류: 소스 작업({source_task_id})을 찾을 수 없거나 시간 내에 완료되지 않았습니다.")

        research_result = source_task.result
        final_report = f"## 최종 보고서 (작성 스타일: {style})\n\n"
        final_report += "### 조사 결과 요약\n"
        final_report += f"{research_result}\n\n"
        final_report += "### 결론\n"
        final_report += "상기 조사 결과를 바탕으로, 해당 주제는 기술적으로 매우 유의미한 발전을 이루고 있으며 향후 지속적인 모니터링이 필요할 것으로 사료됩니다."

        print(f"[{self.worker_id}] ✅ 보고서 작성 완료.")
        return final_report


if "task_registry" in locals() and "redis_client" in locals():
    try:
        redis_client.ping()
        print("--- 🚀 실제 데이터 기반 비동기 협업 시스템 시뮬레이션 시작 ---")

        # 1. Dispatcher와 '전문화된' Worker들 생성
        dispatcher = TaskDispatcher(task_registry, redis_client)
        # WorkerAgent 대신 ResearcherAgent와 WriterAgent를 사용합니다.
        researcher_worker = ResearcherAgent("researcher_01", "research_tasks", task_registry, redis_client)
        writer_worker = WriterAgent("writer_01", "writing_tasks", task_registry, redis_client)

        workers = [researcher_worker, writer_worker]

        try:
            # 2. Worker 스레드들 시작
            for worker in workers:
                worker.start()
            time.sleep(3)

            # 3. Dispatcher가 작업들을 발행
            # Researcher에게 'AI Agent의 미래'에 대한 조사를 지시합니다.
            task1_id = dispatcher.dispatch_task("research_tasks", {"topic": "AI Agent의 미래"}, priority=3)
            time.sleep(1)
            # Writer에게 task1의 결과를 바탕으로 전문적인 스타일의 글을 작성하라고 지시합니다.
            task2_id = dispatcher.dispatch_task(
                "writing_tasks", {"source_task_id": task1_id, "style": "전문적"}, priority=5
            )

            # 4. 작업이 완료될 때까지 대기하며 관찰
            print("\n[Main Thread] ⏳ 작업이 비동기적으로 처리되는 중... (15초 대기)")
            time.sleep(15)

            # 5. Registry에서 최종 결과 확인
            print("\n--- [Main Thread] 📊 최종 결과 확인 ---")
            task1_final = task_registry.get_task(task1_id)
            task2_final = task_registry.get_task(task2_id)

            if task1_final:
                print(f"\n[최종 결과 1: 리서처 작업 - {task1_final.status.value}]")
                print(task1_final.result)
                # AI 에이전트의 현재와 미래에 대한 검색 결과가 출력됩니다. [1, 2, 3, 4, 5]
            if task2_final:
                print(f"\n[최종 결과 2: 작가 작업 - {task2_final.status.value}]")
                print(task2_final.result)

        finally:
            # 6. 모든 Worker 스레드 안전하게 종료
            print("\n[Main Thread] 🛑 모든 워커에게 종료 신호를 보냅니다...")
            for worker in workers:
                worker.stop()
            print("--- ✅ 시뮬레이션 정상 종료 ---")

    except Exception as e:
        print(f"❌ 시뮬레이션 중 오류 발생: {e}")
        import traceback

        traceback.print_exc()
else:
    print("❌ 시스템 초기화에 실패했습니다. 이전 셀들을 다시 실행해주세요.")

### 7. 시스템의 '관제탑': 고급 제어 시스템 구현

지금까지 우리는 독립적으로 작동하는 훌륭한 비동기 시스템을 만들었습니다. 하지만 이 시스템은 아직 '장님' 상태입니다. 어떤 작업이 얼마나 오래 걸리고 있는지, 특정 Worker가 다운되어 작업이 멈춰있는지(데드락), 그리고 더 중요한 작업을 먼저 처리해야 하는지 알지 못합니다.

이번 마지막 파트에서는 이 모든 것을 관리하는 **'관제탑'**을 구현하여, 우리 시스템을 단순한 자동화 도구에서 **신뢰할 수 있는 프로덕션급 워크플로우 엔진**으로 완성합니다.

**구현 목표:**
1.  **Agent 상태 모니터링:** `TaskRegistry`를 주기적으로 스캔하여, 시스템 전체의 작업 현황(대기, 진행 중, 완료 건수)을 실시간으로 파악하는 `SystemMonitor`를 구현합니다.
2.  **데드락 방지 (Timeout 메커니즘):** `RUNNING` 상태로 너무 오래(e.g., 60초 이상) 멈춰있는 '죽은' 작업을 감지하고, 이를 자동으로 `FAILED` 상태로 변경하여 시스템의 안정성을 확보합니다.
3.  **작업 우선순위 관리:** Task에 '우선순위'를 부여하고, Worker Agent들이 항상 가장 중요한 작업을 먼저 처리하도록 Pub/Sub 채널을 분리하고 구독 순서를 제어하는 고급 디스패칭 로직을 구현합니다.

#### 7.1. 시스템 업그레이드: `Task` 모델에 우선순위 추가

가장 먼저, 작업의 중요도를 표현할 수 있도록 `Task` Pydantic 모델을 업그레이드합니다. `priority` 필드를 추가하며, 숫자가 낮을수록 더 높은 우선순위를 의미하도록 설계합니다. (1 = 가장 중요)

In [ ]:
# 이전 파트에서 정의한 클래스들을 가져옵니다.
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any, List
from enum import Enum
import uuid
import datetime


class TaskStatus(str, Enum):
    PENDING = "PENDING"
    RUNNING = "RUNNING"
    COMPLETED = "COMPLETED"
    FAILED = "FAILED"


class Task(BaseModel):
    task_id: str = Field(default_factory=lambda: f"task_{uuid.uuid4()}")
    status: TaskStatus = TaskStatus.PENDING
    payload: Dict[str, Any]
    priority: int = Field(default=5, description="작업 우선순위 (1-10, 낮을수록 높음)")
    result: Optional[str] = None
    worker_id: Optional[str] = None
    created_at: str = Field(default_factory=lambda: datetime.datetime.now(datetime.timezone.utc).isoformat())
    updated_at: str = Field(default_factory=lambda: datetime.datetime.now(datetime.timezone.utc).isoformat())


print("✅ 'priority' 필드가 추가된 Task 모델로 업그레이드되었습니다.")

#### 7.2. 시스템의 '눈': `SystemMonitor` 구현

시스템의 상태를 감시하고, 데드락을 방지하는 `SystemMonitor` 클래스를 구현합니다. 이 모니터 역시 별도의 스레드에서 주기적으로 작동하며 시스템의 건강 상태를 체크합니다.

In [ ]:
import threading
import time
from collections import Counter

# 이전 파트의 TaskRegistry 클래스가 메모리에 있다고 가정합니다.


class SystemMonitor:
    def __init__(self, registry: "TaskRegistry", timeout_seconds: int = 60):
        self.registry = registry
        self.timeout = datetime.timedelta(seconds=timeout_seconds)
        self.stop_event = threading.Event()
        self.thread = threading.Thread(target=self.run, daemon=True)

    def check_for_stuck_tasks(self):
        """'RUNNING' 상태로 너무 오래 멈춰있는 작업을 감지하고 'FAILED'로 변경합니다."""
        all_task_keys = self.registry._client.keys("task_registry:*")
        for key in all_task_keys:
            task_data = self.registry._client.hgetall(key)
            if task_data.get("status") == TaskStatus.RUNNING.value:
                updated_at = datetime.datetime.fromisoformat(task_data["updated_at"])
                if datetime.datetime.now(datetime.timezone.utc) - updated_at > self.timeout:
                    task_id = task_data["task_id"]
                    print(f"[Monitor] 🚨 데드락 감지! Task {task_id}가 {self.timeout.seconds}초 이상 멈춰있습니다.")
                    self.registry.update_task_status(task_id, TaskStatus.FAILED, result="Timeout Error")

    def get_system_snapshot(self) -> dict:
        """시스템의 현재 작업 현황 스냅샷을 반환합니다."""
        all_task_keys = self.registry._client.keys("task_registry:*")
        statuses = [self.registry._client.hget(key, "status") for key in all_task_keys]
        return Counter(statuses)

    def run(self):
        print("[Monitor] 🔭 시스템 모니터링 시작 (10초마다 체크)...")
        while not self.stop_event.is_set():
            self.check_for_stuck_tasks()
            snapshot = self.get_system_snapshot()
            print(f"[Monitor] 📊 현재 상태: {dict(snapshot)}")
            time.sleep(10)
        print("[Monitor] 🛑 모니터링 종료.")

    def start(self):
        self.thread.start()

    def stop(self):
        self.stop_event.set()
        self.thread.join()


print("✅ SystemMonitor 클래스가 성공적으로 정의되었습니다.")

#### 7.3. 시스템 업그레이드: 우선순위 기반 `Dispatcher` 및 `Worker`

마지막으로, Task의 `priority`를 실제로 처리할 수 있도록 `TaskDispatcher`와 `WorkerAgent`를 업그레이드합니다. Dispatcher는 우선순위에 따라 서로 다른 채널에 작업을 발행하고, Worker는 항상 우선순위가 높은 채널을 먼저 확인하도록 만듭니다.

In [ ]:
# --- 채널 정의 ---
HIGH_PRIORITY_CHANNEL = "tasks:priority:high"
LOW_PRIORITY_CHANNEL = "tasks:priority:low"


# --- 업그레이드된 Dispatcher ---
class PriorityTaskDispatcher:
    def __init__(self, registry: "TaskRegistry", redis_conn):
        self.registry = registry
        self.redis_conn = redis_conn

    def dispatch_task(self, payload: Dict[str, Any], priority: int = 5):
        task = self.registry.register_task(payload=payload)
        self.registry.update_task_status(task.task_id, TaskStatus.PENDING, priority=priority)
        channel = HIGH_PRIORITY_CHANNEL if priority < 5 else LOW_PRIORITY_CHANNEL
        notification = {"task_id": task.task_id}
        self.redis_conn.publish(channel, json.dumps(notification))
        print(f"[Dispatcher] 🚚 Task {task.task_id}를 '{channel}' 채널로 전달 (우선순위: {priority})")
        return task.task_id


# --- 업그레이드된 Worker ---
class PriorityWorkerAgent(WorkerAgent):  # 이전 WorkerAgent 클래스를 상속받아 수정
    def __init__(self, worker_id: str, registry: "TaskRegistry", redis_conn):
        # 이 워커는 두 채널을 모두 구독하지만, 높은 우선순위 채널을 먼저 확인합니다.
        super().__init__(worker_id, "priority_worker", registry, redis_conn)
        self.channel_priorities = [HIGH_PRIORITY_CHANNEL, LOW_PRIORITY_CHANNEL]

    def run(self):
        print(f"[{self.worker_id}] 🚀 우선순위 워커 시작. 채널: {self.channel_priorities}")
        pubsub = self.redis_conn.pubsub()
        pubsub.subscribe(*self.channel_priorities)

        while not self.stop_event.is_set():
            message = pubsub.get_message(ignore_subscribe_messages=True, timeout=1.0)
            if message:
                # ... (이전 WorkerAgent의 메시지 처리 로직과 동일) ...
                task_notification = json.loads(message["data"])
                task_id = task_notification["task_id"]
                print(f"[{self.worker_id}] 📬 '{message['channel']}' 채널에서 작업 수신: {task_id}")
                self.registry.update_task_status(task_id, TaskStatus.RUNNING, worker_id=self.worker_id)
                time.sleep(3)
                self.registry.update_task_status(task_id, TaskStatus.COMPLETED, result="작업 완료")

        pubsub.close()
        print(f"[{self.worker_id}] 🛑 워커 종료.")


print("✅ 우선순위 처리가 가능한 Dispatcher와 Worker로 업그레이드되었습니다.")

#### 7.4. 최종 통합 시스템 시뮬레이션

이제 모든 것을 통합하여, 우리의 '관제탑'이 실제로 작동하는 모습을 시뮬레이션합니다.

1.  **SystemMonitor**와 두 명의 **PriorityWorkerAgent**를 백그라운드에서 실행합니다.
2.  Dispatcher가 **낮은 우선순위**의 작업을 먼저 발행합니다.
3.  잠시 후, **높은 우선순위**의 긴급 작업을 발행합니다.
4.  워커들이 어떻게 높은 우선순위의 작업을 먼저 처리하는지, 그리고 전체 시스템의 상태가 모니터에 어떻게 실시간으로 표시되는지 관찰합니다.

In [ ]:
if "task_registry" in locals() and "redis_client" in locals() and redis_client.ping():
    # --- 전체 시스템 시뮬레이션 ---
    print("--- 🚀 최종 통합 시스템 시뮬레이션 시작 ---")

    # 1. 컴포넌트들 생성
    dispatcher = PriorityTaskDispatcher(task_registry, redis_client)
    monitor = SystemMonitor(task_registry, timeout_seconds=20)  # 테스트를 위해 타임아웃 20초
    worker1 = PriorityWorkerAgent("worker_01", task_registry, redis_client)
    worker2 = PriorityWorkerAgent("worker_02", task_registry, redis_client)

    components = [monitor, worker1, worker2]

    try:
        # 2. 모든 백그라운드 스레드 시작
        for component in components:
            component.start()
        time.sleep(2)

        # 3. Dispatcher가 작업들을 발행 (우선순위가 다른 작업)
        dispatcher.dispatch_task({"desc": "일반 로그 분석"}, priority=8)
        dispatcher.dispatch_task({"desc": "데이터베이스 백업"}, priority=7)
        time.sleep(1)
        print("\n*** 🚨 긴급 작업 발생! ***")
        dispatcher.dispatch_task({"desc": "긴급 서버 패치"}, priority=1)

        # 4. 작업이 처리되는 동안 관찰
        print("\n[Main Thread] ⏳ 시스템이 비동기적으로 작동하는 중... (25초 대기)")
        time.sleep(25)

    finally:
        # 5. 모든 컴포넌트 안전하게 종료
        print("\n[Main Thread] 🛑 모든 컴포넌트에 종료 신호를 보냅니다...")
        for component in components:
            component.stop()
        print("--- ✅ 최종 시뮬레이션 정상 종료 ---")
else:
    print("❌ 시스템 초기화에 실패했습니다. 이전 셀들을 다시 실행해주세요.")